# AI エージェントの学習

In [1]:
import os
import openai
from dotenv import load_dotenv
from typing import TypedDict, List, Union

# .envファイルをロード
load_dotenv()

# 環境変数OPENAI_API_KEYを取得
openai.api_key = os.getenv("OPENAI_API_KEY")

# APIキーが設定されているか確認
if openai.api_key is None:
    raise ValueError("OPENAI_API_KEYが設定されていません。")


## 1. AIエージェントのAIエージェントの基本的な概念

AIエージェントを理解するためには、その構成要素となる以下の要素を理解する必要があります

1. 言語モデル
2. プロンプト
3. チェーン
4. エージェント

### 1.1. 言語モデル

LangChainでOpenAIの言語モデルをインスタンス化する基本的なコードは以下の通りです

- OpenAIは、gpt-3.5-turboやgpt-4といった高性能な言語モデルを提供しており、これらは様々な自然言語処理タスクに活用されています
- LangChainは、これらの異なる言語モデルを統一的なインターフェースで利用できる抽象化レイヤーを提供しています
- これにより、開発者は特定のモデルに依存することなく、様々なモデルを試したり、必要に応じて切り替えたりすることが容易になります

In [2]:
#from langchain.llms import OpenAI
#from langchain_community.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

# OpenAIの言語モデルのインスタンスを作成
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

### 1.2. LangChainにおけるPromptTemplate

以下のコードは、`PromptTemplate`を使って、特定のトピックに関するジョークを求めるプロンプトを作成する例です。

- LangChainは、PromptTemplateという便利なクラスを提供しています
- これを利用することで、プレースホルダーを含む動的なプロンプトを作成できます。

In [3]:
from langchain.prompts import PromptTemplate

# プロンプトのテンプレートを作成。{topic}はプレースホルダー
prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

# プレースホルダーに具体的な値を指定してプロンプトを生成
prompt = prompt_template.format(topic="人工知能")
print(prompt)

Tell me a joke about 人工知能


### 1.2. `RunnableSequence` を使って処理のChainを作成する

以下のコードは、`RunnableSequence`を使ってChainを作成し、実行する例です。

- Chainとは、言語モデルへの呼び出しや他のユーティリティを繋ぎ合わせた一連の処理のことです。
- LangChainにおける最も基本的なChainは、プロンプトテンプレートと言語モデルを組み合わせたものです。
- `RunnableSequence`とパイプ演算子 (|) を使用することで、同様の機能を実現できます。これにより、定義されたプロンプトに基づいて言語モデルを呼び出し、その出力を得ることができます
- Chainの概念は、より複雑な処理フローを構築するための基盤となります。


In [4]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# OpenAIのチャットモデルのインスタンスを作成
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# プロンプトのテンプレートを作成。{topic}はプレースホルダー
prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

#
# RunnableSequenceを利用することで、複数のステップからなる処理を簡潔に記述し実行することができます
#

# RunnableSequenceを使ってチェーンを定義
# ここでは、事前に作成した言語モデルのインスタンスllmとプロンプトテンプレートprompt_templateをパイプ演算子 (|) で繋ぎ、チェーンを定義しています
chain = prompt_template | llm

# チェーンを実行し、結果を取得
# そして、invokeメソッドに具体的な入力（ここでは「宇宙探査」）を辞書形式で渡すことで、プロンプトが生成され、言語モデルがそのプロンプトに基づいて応答を生成します。
output = chain.invoke({"topic": "宇宙探査"})
print(output)

content='Why did the astronaut break up with his girlfriend before going on a space mission? \n\nBecause he needed space!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 20, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BbNJG9isPTFzn156FZL80Wafm0bQS', 'finish_reason': 'stop', 'logprobs': None} id='run-25f8ebd7-506a-495f-af16-80af5643c519-0' usage_metadata={'input_tokens': 20, 'output_tokens': 22, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### 1.4. エージェント

LangChainにおけるAIエージェントは、言語モデルを利用して、どのような行動を取るべきかを決定するエンティティです。

- 多くの場合、エージェントはツールと呼ばれる外部の機能を利用して、現実世界とインタラクションしたり、追加の情報を取得したりします
- AIエージェントの核となるのは、言語モデル、利用可能なツールのセット、そしてどのツールをいつ使用するかを決定するロジック（エージェントロジック）です
- LangChainのエージェントは、単に言語モデルに問い合わせるだけでなく、自律的に意思決定を行い、タスクを実行する能力を持つ点が特徴です

## 2. 簡単な質問応答を行うAIエージェントの構築

### 2.1. 最もシンプルなAIエージェントの例

ここでは、LangChainとOpenAIを用いて、簡単な質問応答を行うAIエージェントのサンプルコードを作成します。  
まず、質問を入力として受け取り、それに対する回答を生成する基本的なエージェントを`RunnableSequence`を用いて実装します。

- まず質問応答に特化したプロンプトテンプレートを作成しています
- `{question}`は質問の内容が入るプレースホルダーです
- 次に、OpenAIのチャットモデルをインスタンス化し、作成したプロンプトテンプレートと組み合わせてRunnableSequenceでチェーンを作成します
- 最後に、質問を設定し、invokeメソッドを実行することで、言語モデルが質問に対する回答を生成します

このシンプルな例でも、言語モデルが持つ知識を活用して、基本的な質問応答エージェントとして機能することがわかります。

In [5]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 質問応答のためのプロンプトテンプレートを作成
qa_prompt_template = PromptTemplate.from_template("次の質問に答えてください: {question}")

# OpenAIのチャットモデルをインスタンス化
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# RunnableSequenceを使ってチェーンを定義
qa_chain = qa_prompt_template | llm

# 質問を設定
question = "フランスの首都はどこですか？"

# チェーンを実行して回答を得る
answer = qa_chain.invoke({"question": question})

# 質問と回答を出力
print(f"質問: {question}")
print(f"回答: {answer}")

質問: フランスの首都はどこですか？
回答: content='パリです。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 32, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BbNMIbXVnLwkSj2HcDwDjtJGj9VSC', 'finish_reason': 'stop', 'logprobs': None} id='run-b7ea5a71-f3ba-4aa4-85b4-9684505492bd-0' usage_metadata={'input_tokens': 32, 'output_tokens': 4, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### 2.2. コンテキスト（文脈情報）をプロンプトに含めてみよう

さらに、質問に対する回答の精度と関連性を向上させるために、コンテキスト（文脈情報）をプロンプトに含めることができます。  
以下のコードは、コンテキストに基づいて質問に答えるようにプロンプトテンプレートを修正した例です。

- プロンプトテンプレートに`{context}`というプレースホルダーを追加し、質問に答えるための文脈情報を提供できるようにしています
- `invoke`メソッドを実行する際には、プレースホルダーに対応する値を辞書形式で渡します
- このように明示的にコンテキストを提供することで、言語モデルはより正確で関連性の高い回答を生成することができます
- 特に、特定のドメインや知識に基づいた質問応答を行う場合には、コンテキストの提供が非常に重要になります。

In [6]:
qa_with_context_prompt_template = PromptTemplate.from_template(
    """次の文脈に基づいて質問に答えてください。
    文脈: {context}
    質問: {question}"""
)

# OpenAIのチャットモデルをインスタンス化（前の例でインスタンス化されたものを使用）
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# RunnableSequenceを使ってチェーンを定義
qa_with_context_chain = qa_with_context_prompt_template | llm

# 文脈と質問を設定
context = "フランスの首都はパリです。"
question = "フランスの首都はどこですか？"

# チェーンを実行して回答を得る。プレースホルダーに対応する値を辞書形式で渡す
answer = qa_with_context_chain.invoke({"context": context, "question": question})

# 質問、文脈、回答を出力
print(f"質問: {question}")
print(f"文脈: {context}")
print(f"回答: {answer}")

質問: フランスの首都はどこですか？
文脈: フランスの首都はパリです。
回答: content='パリです。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 66, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BbNMamkz7imtJcLjEmPWaTZKshkkr', 'finish_reason': 'stop', 'logprobs': None} id='run-f0261e07-7ac4-49ce-a1a4-6616e52bec84-0' usage_metadata={'input_tokens': 66, 'output_tokens': 4, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


## 3. ツールを利用して高度なAIエージェントを構築する

より高度なAIエージェントは、外部のツールを利用することで、その能力を大幅に拡張できます。  
ツールとは、エージェントが外部の世界とインタラクションしたり、追加の情報を取得したりするために使用できる機能のことです。例えば、以下のようなツールが考えられます。

- 検索エンジン
- 計算機
- データベース
- API

ツールを利用することで、AIエージェントは単なる言語生成だけでなく、より複雑なタスクを実行できるようになります。
LangChainでは、Toolクラスを抽象化しており、これを利用することで様々なツールを統一的な方法で定義し、エージェントに利用させることができます。  

- Toolは、通常、名前、説明、そして実行するための関数を持ちます
- エージェントは、与えられたユーザーのクエリに基づいて、どのツールが適切かを判断します
- 選択したツールを実行して、得られた結果を基に最終的な応答を生成します

この一連の処理は、エージェントの実行ループと呼ばれます。  
エージェントは、ユーザーからのクエリを受け取り、適切なツールを選択し、実行し、その結果を観察し、最終的な応答を生成するというサイクルを繰り返します。

ここでは、簡単な例として、簡単な計算（足し算）を行うツールを定義し、それを利用するAIエージェントを構築してみましょう。


### 3.1. ツールとLLMの定義

In [7]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import BaseMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.tools import Tool, StructuredTool
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from pydantic import BaseModel, Field # ★追加: PydanticのBaseModelとFieldをインポート

In [ ]:
# pydanticを使用して、ツールの引数スキーマを定義します
class AddInput(BaseModel):
    """Inputs for add function."""
    # 以下のフィールドは、ツールの引数として使用されます。またLLMが適切なToolを選択するためのヒントにもなります
    a: float = Field(description="The first number to add.")
    b: float = Field(description="The second number to add.")

def add(a: float, b: float) -> float:
    """二つの数値を足します。"""
    # 実行ログを追加
    print(f"\n--- Executing Tool: Simple_Calculator with inputs a={a}, b={b} ---")
    return a + b

# StructuredTool を使用します
calculator_tool = StructuredTool(
    name="Simple_Calculator",
    func=add,                           # 先ほど定義した関数を指定
    description="2つの数値を足します。",    # ツールの説明
    args_schema=AddInput                # 引数スキーマを指定  
)

tools = [calculator_tool]
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

### 3.2. 状態の定義

以下はAIエージェントの **「現在の状態（おぼえていること）」を定義している**部分

- エージェントは、次に何をすべきか判断するために、これまでの会話や途中の結果など、いろいろなことを覚えておく必要がある
- この **「おぼえていること」を入れる箱の設計図**が `AgentState` 
- `operator.add`: LangGraph のシステムに対して、「もし誰か（ノード）が新しいメッセージを返してきたら、その新しいメッセージを、この『会話の記録（messages）』リストの一番最後にそのままつけ加えなさい」というルールを伝えている
- `TypedDict`:『現在の状態』という箱には、『messages』という『会話のきろくリスト』が入るよ」という設計図を作っています。
- `messages: Sequence[BaseMessage]` : 「会話の記録」 は、メッセージを順番に並べたリストという型を示している
- `Annotated[..., operator.add]` :「会話の記録に新しいメッセージをリストの一番最後に付け加えるという特別なルールがあるよ」という指示をLangGraphに与えている
  - operator.add がリストにとっての「足し算＝結合」を意味するため、このような動きになる

In [9]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

### 3.3. ノード関数の定義 (変更なし)

In [10]:
def call_llm(state: AgentState):
    """現在の状態（メッセージ履歴）に基づいてLLMを呼び出し、応答を生成します。"""
    messages = state['messages']
    print(f"\n--- Calling LLM with messages: ---\n{messages}\n--- End of messages ---")

    # LLMに利用可能なツールを伝えます。
    # args_schema を定義したツールを使うことで、LLMは引数の構造を理解しやすくなります。
    # ツールの定義をLLMに渡すことで、LLMはどのツールを使うべきかを判断します。複数のツールがある場合、LLMは最も適切なツールを選択します。
    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke(messages)

    print(f"--- LLM Response: ---\n{response}\n--- End of LLM Response ---")
    return {"messages": [response]}

### 3.4. グラフの構築 (変更なし) 

In [11]:
# StateGraph をインスタンス化し、エージェントの状態（記憶やデータ）の型として AgentState を指定します。
# AgentState で定義された構造（今回は messages リスト）が、グラフの各ノード間を流れ、更新されていきます。
workflow = StateGraph(AgentState)

# グラフにノード（処理のステップ）を追加します。
# "llm" という名前で、事前に定義した call_llm 関数（LLM呼び出しロジック）を実行するノードを登録します。
# グラフがこの "llm" ノードに到達すると、call_llm 関数が現在の状態（state）を受け取って実行されます。
workflow.add_node("llm", call_llm)

# もう一つノードを追加します。
# "action" という名前で、ToolNode のインスタンスを登録します。
# ToolNode は LangGraph に組み込まれた便利なノードで、LLMが生成したツール呼び出しを自動的に解釈し、
# コンストラクタに渡されたツールリスト (tools) の中から該当するツールを実行します。
# グラフがこの "action" ノードに到達すると、前のノード（通常はLLM）からの出力を見てツール実行が必要か判断し、実行します。
workflow.add_node("action", ToolNode(tools))

# グラフの実行開始地点を設定します。
# グラフを起動した際に、最初にどのノードから処理を開始するかを指定します。
# この場合、実行は "llm" ノードから始まり、LLMがユーザー入力に対して最初の応答や判断を行います。
workflow.set_entry_point("llm")

# ※ この後、add_conditional_edges や add_edge を使って、
#    各ノードの実行後に「次にどのノードへ進むか」という繋がり（エッジ）を定義します。

### 3.5. エッジ（ノード間の遷移）の定義

グラフのノード間の遷移を定義します。LLMがツールを呼び出したら 'action'へ、最終応答なら終了、という流れを定義しています。

#### 条件付きエッジの追加 (条件分岐を定義できる)

これは、特定のノードの実行結果（現在の状態）に基づいて、次に実行するノードを動的に決定するための重要な部分です。

- `"llm"` 
	- この条件分岐の「出発ノード」を指定
    - "llm" ノードの処理が完了した後に、どのノードへ進むかをこの定義で判断する
- `lambda state: "action" if state['messages'][-1].tool_calls else END`
	- 現在のエージェントの「状態」(state) を受け取り、次に進むべきノードの名前を指定する。ここでは以下のどちらかが選択される
		- "action": 足し算を行う
		- "END": グラフの終了を示す特別な値

**lamda関数の詳細**

- `state['messages'][-1]`: 
	- 現在の状態が持つメッセージリストの中の**一番最後のメッセージ**を取得
	- このケースでは "llm" ノードからの出力（LLMの応答）
- `.tool_calls`: 
	- その「一番最後のメッセージ」（LLMの応答）に、LLM がツールを呼び出すよう指示する情報 **tool_calls オブジェクト**が含まれているかどうかを示す属性
	- ツール呼び出し指示があれば `True` になり、なければ `None` になる
- `state['messages'][-1].tool_calls`
	- `True` であれば文字列 "action" が返却される
		- これにより、グラフは次に "action" ノード（ツール実行ノード）へ遷移する
	- その他の場合、例えばLLMが最終的な回答を生成した場合などLangGraph の特殊な値 `END` が返され、グラフの実行は終了する
		- `END` は、LangGraph の特別な値で、グラフの実行を終了することを示す

In [12]:
workflow.add_conditional_edges(
    "llm", # 出発ノード
    lambda state: "action" if state['messages'][-1].tool_calls else END, # 条件と次のノード/終了
)

**補足**: 上記の2つ目の引数は以下のような関数オブジェクトでもOK, 今回はlambda関数を利用している

```python
def determine_next_node(state):
    """
    現在の状態を受け取り、LLM応答に基づいて次に進むノードを判断する関数。
    LLMの最後のメッセージにtool_callsがあれば'action'を、なければENDを返す。
    """
    # 状態から最後のメッセージを取得
    last_message = state['messages'][-1]
    # 最後のメッセージに tool_calls 属性があり、かつそれが空でなければ（ツール呼び出しがあれば）
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
         return "action" # 'action' ノードに進む
    # そうでなければ（最終応答など）
    else:
        return END # グラフの実行を終了する
```

#### 無条件エッジの追加 (Aノードを処理したら次に必ずXに進んでを定義)

特定のノードの実行が完了した後に、**常に次の指定したノードへ遷移**するためのもの。  
例えば,Aを処理したら必ずXに進んでねを定義できる

- `'action'`: この遷移の「出発ノード」を指定。上記説明のAに相当
- `'llm'`: この遷移の「到達ノード」を指定。上記説明のXに相当

つまり`action`ノードの実行後は、常に`llm`ノードへ進む

- ツール実行ノードはツールを実行し、その結果（ToolMessage）を状態に追加する
- そのツール結果を見た上で、LLM に「次に何をすべきか」（さらに別のツールを使うか、それとも最終回答を生成するかなど）を判断させる必要があるから

これにより、「LLMで考える → ツールで実行 → 結果を見てLLMでまた考える → ...」という ReAct のようなループ構造が生まれます。

In [13]:
workflow.add_edge('action', 'llm')

```mermaid
graph TD
    %% Start node
    start_node(("Start<br/>(workflow.set_entry_point)"));

    %% Entry point: From Start to 'llm' node
    start_node --> llm_node;

    %% Node definitions and transitions
    llm_node["Node 'llm'<br/>(call_llm function)"] --> check_condition{Does the last LLM response<br/>contain tool_calls?};

    check_condition --> |"Yes<br/>(tool_calls exist)"| action_node["Node 'action'<br/>(ToolNode(tools))"];

    check_condition --> |"No<br/>(no tool_calls)"| end_node((End));

    %% Unconditional transition from 'action' node back to 'llm' node
    action_node --> llm_node;

    %% End node
    end_node((End));
```

### 3.6. グラフのコンパイル (変更なし)

In [14]:
# StateGraphを使用してワークフローを定義
app = workflow.compile()

### 3.7. 実行例1

In [16]:
# エージェントに実行させる入力メッセージ
inputs = {"messages": [HumanMessage(content="10と25を足してくれますか？")]}

print("\n--- Invoking the agent ---")
# 定義したグラフを実行します
# verbose=True は invoke 自体の詳細なログは出しません（ノード内のprintは出ます）
# stream を使うと、各ノードの出力を逐次確認できます。
for step in app.stream(inputs, {"recursion_limit": 150}): # recursion_limitはループ回数の上限
    print(f"--- Step Output: {step}---")
    for key, value in step.items():
        print(f"Node: {key}")
        # print(f"State: {value}") # 状態全体を表示すると冗長になるためコメントアウト
    print("-" * 20)

print("--- Agent finished ---")


--- Invoking the agent ---

--- Calling LLM with messages: ---
[HumanMessage(content='10と25を足してくれますか？', additional_kwargs={}, response_metadata={})]
--- End of messages ---
--- LLM Response: ---
content='' additional_kwargs={'tool_calls': [{'id': 'call_BbOvtCV85eRRqlIrP5znbhIJ', 'function': {'arguments': '{"a":10,"b":25}', 'name': 'Simple_Calculator'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 83, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BbNnQG0wT5e7UhQfotRhSOHXjVXgo', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-b48ba7c9-98ad-48ef-8f1a-6f155039fb63-0' tool_calls=[{'name': 'Simple_Calculator', 'args': {'a': 10, 'b': 25}, 'id': 'call_BbOvtC

### 3.8. 実行例2

In [17]:
# 別の実行例（ツールを使わないケース）
inputs_2 = {"messages": [HumanMessage(content="こんにちは！")]}
print("\n--- Invoking the agent (Greeting) ---")
for step in app.stream(inputs_2, {"recursion_limit": 150}):
     print("--- Step Output ---")
     for key, value in step.items():
         print(f"Node: {key}")
         # print(f"State: {value}")
     print("-" * 20)
print("--- Agent finished ---")


--- Invoking the agent (Greeting) ---

--- Calling LLM with messages: ---
[HumanMessage(content='こんにちは！', additional_kwargs={}, response_metadata={})]
--- End of messages ---
--- LLM Response: ---
content='こんにちは！どのようにお手伝いしましょうか？' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 73, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BbNndsWYGwFF9zhgOPd9zMPfJU3YP', 'finish_reason': 'stop', 'logprobs': None} id='run-c77a5013-d9c1-48f2-960b-3cadd5c1cd52-0' usage_metadata={'input_tokens': 73, 'output_tokens': 21, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
--- End of LLM Response ---
--- Step O

## 4. 掛け算も可能にしよう！

In [18]:
# pydanticを使用して、ツールの引数スキーマを定義します
class AddInput(BaseModel):
    """Inputs for add function."""
    # 以下のフィールドは、ツールの引数として使用されます。またLLMが適切なToolを選択するためのヒントにもなります
    a: float = Field(description="The first number to add.")
    b: float = Field(description="The second number to add.")

class MultInput(BaseModel):
    """Inputs for add function."""
    # 以下のフィールドは、ツールの引数として使用されます。またLLMが適切なToolを選択するためのヒントにもなります
    a: float = Field(description="The first number to mult.")
    b: float = Field(description="The second number to mult.")

def add(a: float, b: float) -> float:
    """二つの数値を足します。"""
    # 実行ログを追加
    print(f"\n--- {a} + {b} ---")
    return a + b

def mult(a: float, b: float) -> float:
    """二つの数値を掛け合わせます。"""
    # 実行ログを追加
    print(f"\n--- {a} * {b} ---")
    return a * b

# StructuredTool を使用します
calculator_tool_add = StructuredTool(
    name="Simple_Calculator_add",
    func=add,                           # 先ほど定義した関数を指定
    description="2つの数値を足します。",    # ツールの説明
    args_schema=AddInput               # 引数スキーマを指定  
)

calculator_tool_mult = StructuredTool(
    name="Simple_Calculator_mult",
    func=mult,                           # 先ほど定義した関数を指定
    description="2つの数値を乗算します。",    # ツールの説明
    args_schema=MultInput                 # 引数スキーマを指定  
)

tools = [calculator_tool_add, calculator_tool_mult]
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [ ]:
def call_llm(state: AgentState):
    """現在の状態（メッセージ履歴）に基づいてLLMを呼び出し、応答を生成します。"""
    messages = state['messages']
    print(f"\n--- Calling LLM with messages: ---\n{messages}\n--- End of messages ---")
    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke(messages)
    print(f"--- LLM Response: ---\n{response}\n--- End of LLM Response ---")
    return {"messages": [response]}

In [ ]:
workflow = StateGraph(AgentState)
workflow.add_node("llm", call_llm)
workflow.add_node("action", ToolNode(tools))
workflow.set_entry_point("llm")
workflow.add_conditional_edges(
    "llm", # 出発ノード
    lambda state: "action" if state['messages'][-1].tool_calls else END, # 条件と次のノード/終了
)
workflow.add_edge('action', 'llm')

In [ ]:
# StateGraphを使用してワークフローを定義
app = workflow.compile()

In [ ]:
# エージェントに実行させる入力メッセージ
#inputs = {"messages": [HumanMessage(content="10と25を足してください")]}
inputs = {"messages": [HumanMessage(content="10に25を掛けてください")]}

print("\n--- Invoking the agent ---")
# 定義したグラフを実行します
# verbose=True は invoke 自体の詳細なログは出しません（ノード内のprintは出ます）
# stream を使うと、各ノードの出力を逐次確認できます。
for step in app.stream(inputs, {"recursion_limit": 150}): # recursion_limitはループ回数の上限
    print(f"--- Step Output: {step}---")
    for key, value in step.items():
        print(f"Node: {key}")
        # print(f"State: {value}") # 状態全体を表示すると冗長になるためコメントアウト
    print("-" * 20)

print("--- Agent finished ---")

# Appendix: 理解を助けるためのサンプルコード

## LLMがどのように適正に使用するツールを選択するのか？

LLM が登録されたツールを適切に選択し、そして正しく呼び出すために最も重要な情報は、実は**複数あり、それぞれが異なる役割を果たします。**

ご質問の `description="2つの数値を足します。"` は、確かに**非常に重要な情報の一つ**です。これは LLM に対して、そのツールが「何を達成するためのものか」を自然言語で伝える、ツールの**目的の要約**だからです。LLM はユーザーの要求とツールの説明を比較して、「ユーザーの『足してほしい』という要求には、この『2つの数値を足す』ツールが合っているな」と判断します。

しかし、目的が分かっただけでは、LLM はツールを**正しく呼び出す**ことができません。ツールを呼び出すには、**何という名前で呼び出すか**、そして**どのような情報を引数として渡すか**を知る必要があります。

ここで重要になるのが：

1.  **ツールの名前 (`name="Simple_Calculator"`)**: LLM が「このツールを使おう」と決めたときに、その判断をシステムに伝えるための固有の識別子です。
2.  **`args_schema` (Pydantic モデル `AddInput` とその中の `Field` 定義)**: これが、ツールが**「どんな種類の情報（引数）を、どのような名前で、いくつ必要としているか」**を伝える構造化された情報です。
    * `args_schema` に含まれる **引数名** (`a`, `b`)：LLM はこれで、「このツールには `a` と `b` という名前の情報が必要なんだな」と理解します。
    * `args_schema` に含まれる **引数の型** (`float`)：LLM はこれで、「`a` と `b` には数値（小数点を含む可能性のある数）を渡せばいいんだな」と理解します。
    * `args_schema` の `Field` に含まれる **引数の説明** (`description="The first number to add."` など)：これは**非常に重要です。**LLM はこの説明を見て、「`a` というのは『足し算する一つ目の数』のことか」「`b` は『二つ目の数』のことか」と理解し、ユーザーのプロンプトの中からどの部分を `a` に、どの部分を `b` に当てはめれば良いかを判断します。

**結論として：**

* ツール全体の **`description`** は、LLM が「このツールは関連しているか？」と判断するための**入口（目的の理解）**として非常に重要です。
* `args_schema` 全体（特に**引数名**と**引数の `description`**）は、LLM が「このツールを使うためには何が必要か？」「ユーザーの要求から、必要な情報はどこにあるか？」「どのような形式で情報を渡せば良いか？」と判断するための**具体的な手順書**として非常に重要です。

どちらか一方だけでは不十分です。たとえるなら、

* 全体の `description` は「これは料理を作る道具です（包丁です）」という説明です。
* `args_schema` は「材料を切るには、『切るもの（野菜）』と『切り方（薄切り/みじん切りなど）』が必要です」という具体的な使い方と必要な情報の説明です。

LLM がツールを適切に選択し、そしてエラーなく実行させるためには、**ツール全体の `description`** と、`args_schema` 内の**各引数の `description`** の両方が、**明確で正確であること**が最も重要と言えます。どちらも欠かせない要素です。

## TypeDictの理解

### テストコード

In [ ]:
from typing import TypedDict

# 「ユーザー情報」という辞書のレシピ
class UserInfo(TypedDict):
    name: str   # 'name'というキーがあり、値は文字列(str)
    age: int    # 'age'というキーがあり、値は整数(int)
    is_active: bool # 'is_active'というキーがあり、値は真偽値(bool)

### 正常系のテスト

In [ ]:
# このレシピに沿った辞書
user1: UserInfo = {"name": "山田", "age": 30, "is_active": True}
user1

### 異常系のテスト

ここではメッセージはでない.MyPy等を使って実行すると警告を得ることができる

In [ ]:
# レシピに合わない辞書 (ageが文字列なので、型ヒントとしてはエラー)
user2: UserInfo = {"name": "田中", "age": "25", "is_active": False} # 型チェッカーは警告を出す
print(user2)

## Pydanticで確実にエラーを出させる

### テストコード

In [ ]:
# 必要なものをPydanticからインポート
from pydantic import BaseModel, Field, ValidationError

# TypedDict の代わりに BaseModel を継承する ★変更点★
# クラス名は TypedDict 版と区別するために UserInfoPydantic とします
class UserInfoPydantic(BaseModel):
    """ユーザー情報のPydanticモデル"""
    # フィールド名、型ヒント、Fieldを使った設定を記述
    name: str = Field(description="ユーザーの名前")
    age: int = Field(description="ユーザーの年齢")
    is_active: bool = Field(description="ユーザーがアクティブかどうか")

### 正常系の動作

In [ ]:
# --- Pydanticモデルの使い方（ランタイム検証のデモ） ---

print("--- 正しいデータでモデルを作成 ---")
try:
    # 定義に合う正しいデータ（辞書）を渡してモデルのインスタンスを作成
    user1_data = {"name": "山田", "age": 30, "is_active": True}
    user1_instance = UserInfoPydantic(**user1_data) # 辞書を展開して渡す
    # または user1_instance = UserInfoPydantic.model_validate(user1_data) # こちらの書き方もある

    print("モデル作成成功:")
    print(user1_instance)
    print(f"名前: {user1_instance.name}, 年齢: {user1_instance.age}") # インスタンスから属性としてアクセス可能

except ValidationError as e:
    print("モデル作成に失敗しました (予期しないエラー):", e)

### 入力の型は間違っているがPydanticの自動補正機能が働いたケース

In [ ]:
print("\n--- 間違ったデータでモデルを作成（エラー発生のデモ） ---")
try:
    # 定義に合わない間違ったデータ（ageが文字列）
    user2_data_invalid = {"name": "田中", "age": "25", "is_active": False}

    # 間違ったデータを渡してモデルのインスタンスを作成 ★ここでエラーが発生します★
    user2_instance = UserInfoPydantic(**user2_data_invalid)

    print("モデル作成成功 (このメッセージは表示されないはず):")
    print(user2_instance)

except ValidationError as e:
    # ValidationError が捕捉される
    print("モデル作成に失敗しました (期待通り):")
    print(e) # どんなエラーが発生したか詳細が表示される

print("\n--- 終了 ---")

### 自動補正が効かないわざとエラーがでるケース

In [ ]:
# --- 間違ったデータでモデルを作成（エラー発生のデモ） ---
print("\n--- 間違ったデータでモデルを作成（エラー発生のデモ） ---")
try:
    # 定義に合わない間違ったデータ（ageが文字列、かつ整数に変換できないもの） ★ここを変更★
    user2_data_invalid = {"name": "田中", "age": "二十五", "is_active": False} # 例: 整数に変換できない文字列
    # または user2_data_invalid = {"name": "田中", "age": [25], "is_active": False} # 例: リスト

    # 間違ったデータを渡してモデルのインスタンスを作成 ★ここでValidationErrorが発生します★
    user2_instance = UserInfoPydantic(**user2_data_invalid)

    print("モデル作成成功 (このメッセージは表示されないはず):")
    print(user2_instance)

except ValidationError as e:
    # ValidationError が捕捉される ★今度はここが実行されます★
    print("モデル作成に失敗しました (期待通り):")
    print(e) # どんなエラーが発生したか詳細が表示される

print("\n--- 終了 ---")

## `from typing import Annotated` の機能

`Annotated` は、「**いつもの型ヒントに、あとから別の情報をくっつける**」ためのものです。

たとえるなら、変数につける「これは数字です」「これは単語です」「これはリストです」という**いつものラベル**に、**特別な「付箋（ふせん）」をペタッと貼り付ける**ようなイメージです。

図で見てみましょう。

まず、普通の型ヒントのラベルです。

```terminal
+-----------------+
|  Variable Box   |
+-----------------+
|                 |
|                 |
|   (Contents)    |
|                 |
+-----------------+
     |
     |  This is...
     v
+--------------+
| [Type Hint]  |  ← Usual Label (e.g., int, str, List[something])
+--------------+
```

`Annotated` を使うと、この「いつものラベル」に、さらに別の「付箋」情報をくっつけられます。

```terminal
+-----------------+
|  Variable Box   |
+-----------------+
|                 |
|                 |
|   (Contents)    |
|                 |
+-----------------+
     |
     |  This is...
     v
+--------------+   +------------------------+
| [Type Hint]  | + | ★Special Sticky Note ★ |  ← Annotated allows adding this note
+--------------+   |                        |
                   |   Any info you want    |
                   |      can be written    |
                   +------------------------+
```

コードの `Annotated[Sequence[BaseMessage], operator.add]` の場合、これはこういう意味になります。

1.  **いつものラベル**: `Sequence[BaseMessage]` (メッセージのリストです)
2.  **特別な付箋**: `operator.add` (リストを「足し合わせる」、つまり結合するという情報)

これを変数 `messages` の型ヒントとして使うと...

```terminal
+-------------------------+
|      messages Box       |
+-------------------------+
|                         |
|  [Message 1]            |
|  [Message 2]            |
|  ...                    |
+-------------------------+
     |
     |  This is...
     v
+---------------------------+   +-----------------------+
| [It's a List of Messages] | + | ★Special Sticky Note★ |  ← Added by Annotated
+---------------------------+   |                       |
                                |  [Adding Rule]        |  ← The info from operator.add
                                +-----------------------+

```

ここで重要なのは、この「特別な付箋」に書かれた情報は、**Python が普通にコードを実行するときには、ほとんど無視される**ということです。Python は「メッセージのリストだな」ということだけを見て動きます。

しかし、**LangGraph のような「この特別な付箋の読み方を知っている」プログラムやライブラリ**は、この付箋を見つけて書かれた情報を利用します。

LangGraph は `messages` の型ヒントに `Annotated` と `operator.add` が付いているのを見ると、「なるほど、この `messages` のリストには、『新しい情報が来たら前の情報に足し合わせる』というルールが設定されているんだな」と理解します。そして、ノードが新しいメッセージのリストを返したとき、その「足し合わせるルール」に従って、元々 `messages` に入っていたリストの**後ろに**新しいリストをくっつける（結合する）という処理を実行します。

まとめ：

* `Annotated` は、変数に付ける「型」というラベルに、**追加の「特別な情報（付箋）」をペタッと貼り付ける**ためのものです。
* この付箋の情報は、**標準のPythonは気にしません**。
* しかし、**特別なプログラム（LangGraphなど）は、この付箋を読んで、いつもとは違う特別な動きをする**ように作られています。
* `Annotated[Type, metadata]` は、「この変数は `Type` という種類だけど、特別な情報として `metadata` も付いているよ」という意味になります。

「このデータは〇〇だよ」という情報に、「△△する時にこの情報を参考にしてね」という補足情報や指示を付ける、と考えれば分かりやすいかもしれません。